In [19]:
import os
import sys
import tempfile
import urllib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,Lambda
from tensorflow.keras import Sequential
from keras.regularizers import l2

from tensorflow.keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import numpy as np
import pandas as pd
from google.colab import files


In [3]:
gcs_base_dir = "gs://celeb_a_dataset/"
celeb_a_builder = tfds.builder("celeb_a", data_dir=gcs_base_dir, version='2.0.0')

celeb_a_builder.download_and_prepare()

num_test_shards_dict = {'0.3.0': 4, '2.0.0': 2} # Used because we download the test dataset separately
version = str(celeb_a_builder.info.version)
print('Celeb_A dataset version: %s' % version)

Celeb_A dataset version: 2.0.0


In [4]:
local_root = tempfile.mkdtemp(prefix='test-data')
def local_test_filename_base():
  return local_root

def local_test_file_full_prefix():
  return os.path.join(local_test_filename_base(), "celeb_a-test.tfrecord")

def copy_test_files_to_local():
  filename_base = local_test_file_full_prefix()
  num_test_shards = num_test_shards_dict[version]
  for shard in range(num_test_shards):
    url = "https://storage.googleapis.com/celeb_a_dataset/celeb_a/%s/celeb_a-test.tfrecord-0000%s-of-0000%s" % (version, shard, num_test_shards)
    filename = "%s-0000%s-of-0000%s" % (filename_base, shard, num_test_shards)
    res = urllib.request.urlretrieve(url, filename)

In [5]:
ATTR_KEY = "attributes"
IMAGE_KEY = "image"
LABEL_KEY = "Male"
IMAGE_SIZE = 256

In [6]:
def preprocess_input_dict(feat_dict):
  # Separate out the image and target variable from the feature dictionary.
  image = feat_dict[IMAGE_KEY]
  label = feat_dict[ATTR_KEY][LABEL_KEY]

  # Resize and normalize image.
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
  image /= 255.0

  # Cast label and group to float32.
  label = tf.cast(label, tf.float32)

  feat_dict[IMAGE_KEY] = image
  feat_dict[ATTR_KEY][LABEL_KEY] = label

  return feat_dict

get_image_and_label = lambda feat_dict: (feat_dict[IMAGE_KEY], feat_dict[ATTR_KEY][LABEL_KEY])

In [7]:
# Train data returning either 2 elements
def celeb_a_train_data(batch_size):
  celeb_a_train_data = celeb_a_builder.as_dataset(split='train').shuffle(1024).repeat().batch(batch_size).map(preprocess_input_dict)
  return celeb_a_train_data.map(get_image_and_label)

# Train data returning either 2 elements
def celeb_a_val_data(batch_size):
  celeb_a_val_data = celeb_a_builder.as_dataset(split='validation').shuffle(1024).repeat().batch(batch_size).map(preprocess_input_dict)
  return celeb_a_val_data.map(get_image_and_label)

# Test data for the overall evaluation
celeb_a_test_data = celeb_a_builder.as_dataset(split='test').batch(1).map(preprocess_input_dict).map(get_image_and_label)
# Copy test data locally
copy_test_files_to_local()

In [8]:
learning_rate=0.001
num_epochs = 150
num_neurons=256
b_size=32

In [9]:
def step_decay(epoch):
    init_lrate = 1e-3 #TOCHANGE
    drop = 0.1
    epochs_drop = 10000
    lrate = init_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
    
#Definition of weight initializers, optimizers, loss function and learning rate
weight_init = keras.initializers.TruncatedNormal(mean=0.0,stddev=0.01,seed=10)
#bias_init = tf.keras.initializers.Constant(value=0.1)
sgd = keras.optimizers.SGD(learning_rate=0.001,momentum=0.0) #TOCHANGE
loss_func = keras.losses.BinaryCrossentropy()
lrate = keras.callbacks.LearningRateScheduler(step_decay)

In [ ]:
gender_model = keras.models.Sequential([
      Conv2D(96, (7,7), input_shape=(256,256,3), strides=4, padding='valid', activation='relu', kernel_initializer = weight_init),
      MaxPooling2D(pool_size = (3, 3), strides = 2, padding='same'),
      Lambda(lambda x: tf.nn.local_response_normalization(input=x, alpha=0.0001, beta=0.75)),

      Conv2D(256, (5,5), padding='same', activation = 'relu', kernel_initializer = weight_init),
      MaxPooling2D(pool_size = (3, 3), strides = 2, padding = 'same'),
      Lambda(lambda x: tf.nn.local_response_normalization(input=x, alpha=0.0001, beta=0.75)),

      Conv2D(384,(3,3), padding='same', activation='relu', kernel_initializer = weight_init),
      MaxPooling2D(pool_size=(3,3), strides = 2, padding='same'),
      Flatten(),

      Dense(512, activation = "relu", kernel_initializer = weight_init),
      Dropout(0.5),

      Dense(512, activation='relu',kernel_initializer = weight_init),
      Dropout(0.5),

      Dense(1, activation ='sigmoid', kernel_initializer = weight_init)
])

gender_model.compile(loss = loss_func, optimizer = sgd, metrics=['accuracy'])


In [11]:
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)

In [14]:
checkpoint_filepath = "/content/checkpoints/celebA_pretraining_"+str(b_size)+"_"+str(learning_rate)+"_"+str(num_neurons)+".tf"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=checkpoint_filepath,
monitor='val_loss',
mode='min',
save_best_only=True)
early_stopping=EarlyStopping(monitor='val_loss',patience=10)
history = gender_model.fit(celeb_a_train_data(b_size),
                    epochs=num_epochs,
                    steps_per_epoch=1000,
                    verbose=1,
                    use_multiprocessing=True,
                    callbacks=[model_checkpoint_callback],
                    validation_data=celeb_a_val_data(b_size),
                    validation_steps = 1000)

Epoch 1/150
 999/1000 [============================>.] - ETA: 0s - loss: 0.6800 - accuracy: 0.5809INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


1000/1000 [==============================] - 53s 53ms/step - loss: 0.6799 - accuracy: 0.5810 - val_loss: 0.6817 - val_accuracy: 0.5755
Epoch 2/150
1000/1000 [==============================] - 52s 52ms/step - loss: 0.6803 - accuracy: 0.5799 - val_loss: 0.6818 - val_accuracy: 0.5751
Epoch 3/150
1000/1000 [==============================] - 50s 50ms/step - loss: 0.6795 - accuracy: 0.5824 - val_loss: 0.6820 - val_accuracy: 0.5747
Epoch 4/150
1000/1000 [==============================] - 51s 51ms/step - loss: 0.6800 - accuracy: 0.5808 - val_loss: 0.6819 - val_accuracy: 0.5749
Epoch 5/150
1000/1000 [==============================] - ETA: 0s - loss: 0.6802 - accuracy: 0.5800INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


1000/1000 [==============================] - 51s 51ms/step - loss: 0.6802 - accuracy: 0.5800 - val_loss: 0.6816 - val_accuracy: 0.5759
Epoch 6/150
1000/1000 [==============================] - 49s 49ms/step - loss: 0.6805 - accuracy: 0.5792 - val_loss: 0.6820 - val_accuracy: 0.5746
Epoch 7/150
1000/1000 [==============================] - 50s 50ms/step - loss: 0.6797 - accuracy: 0.5815 - val_loss: 0.6817 - val_accuracy: 0.5755
Epoch 8/150
1000/1000 [==============================] - 50s 50ms/step - loss: 0.6794 - accuracy: 0.5825 - val_loss: 0.6818 - val_accuracy: 0.5751
Epoch 9/150
1000/1000 [==============================] - 51s 51ms/step - loss: 0.6801 - accuracy: 0.5805 - val_loss: 0.6819 - val_accuracy: 0.5748
Epoch 10/150
1000/1000 [==============================] - 50s 50ms/step - loss: 0.6807 - accuracy: 0.5786 - val_loss: 0.6818 - val_accuracy: 0.5753
Epoch 11/150
1000/1000 [==============================] - 50s 50ms/step - loss: 0.6801 - accuracy: 0.5803 - val_loss: 0.6819 - va

...

1000/1000 [==============================] - 51s 51ms/step - loss: 0.1503 - accuracy: 0.9403 - val_loss: 0.1210 - val_accuracy: 0.9523
Epoch 149/150
 999/1000 [============================>.] - ETA: 0s - loss: 0.1455 - accuracy: 0.9435INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


1000/1000 [==============================] - 51s 51ms/step - loss: 0.1456 - accuracy: 0.9436 - val_loss: 0.1194 - val_accuracy: 0.9531
Epoch 150/150
1000/1000 [==============================] - ETA: 0s - loss: 0.1438 - accuracy: 0.9441INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


INFO:tensorflow:Assets written to: /content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets


1000/1000 [==============================] - 52s 52ms/step - loss: 0.1438 - accuracy: 0.9441 - val_loss: 0.1178 - val_accuracy: 0.9532


In [15]:
results = gender_model.evaluate(celeb_a_test_data)

19962/19962 [==============================] - 77s 4ms/step - loss: 0.1348 - accuracy: 0.9453


In [18]:
loss=[]
accuracy=[]
val_loss=[]
val_accuracy=[]
loss=history.history['loss'].copy()
val_loss=history.history['val_loss'].copy()
val_accuracy=history.history['val_accuracy'].copy()
accuracy=history.history['accuracy'].copy()

path_1="/content/csvlog/csvlog.csv"
df_eval=pd.DataFrame(list(zip(loss,val_loss,val_accuracy,accuracy)),columns=['loss','val_loss','val_accuracy','accuracy'])
df_eval.to_csv(path_1 ,index=False)

In [20]:
!zip -r /content/baseline_gender_checkpoints.zip /content/checkpoints
files.download("/content/baseline_gender_checkpoints.zip")

!zip -r /content/baseline_gender_csvlog.zip /content/csvlog
files.download("/content/baseline_gender_csvlog.zip")

  adding: content/checkpoints/ (stored 0%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/ (stored 0%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/variables/ (stored 0%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/variables/variables.index (deflated 57%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/assets/ (stored 0%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/keras_metadata.pb (deflated 92%)
  adding: content/checkpoints/celebA_pretraining_32_0.001_256.tf/saved_model.pb (deflated 89%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: content/csvlog/ (stored 0%)
  adding: content/csvlog/csvlog.csv (deflated 59%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>